In [1]:
import pandas as pd
import numpy as np
from pymatgen.core import Composition
import re
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
from IPython.core.pylabtools import figsize
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
import warnings
warnings.filterwarnings("ignore")

In [ ]:
small_dataset = pd.read_csv('/ata/single_perovskite_oxide.csv')
small_dataset = small_dataset.drop(columns = 'Unnamed: 0').reset_index(drop=True)
small_dataset = small_dataset.drop(columns = 'composition').reset_index(drop=True)
small_dataset['composition'] = small_dataset['formula'].map(Composition)
small_dataset

,formula,target,A,B,composition
0,SmCrO3,1.736200,Sm,Cr,"(Sm, Cr, O)"
1,ScCoO3,0.786000,Sc,Co,"(Sc, Co, O)"
2,GdCrO3,0.825000,Gd,Cr,"(Gd, Cr, O)"
3,YCrO3,1.767900,Y,Cr,"(Y, Cr, O)"
4,LaFeO3,0.848400,La,Fe,"(La, Fe, O)"
5,SmCoO3,0.803900,Sm,Co,"(Sm, Co, O)"
6,YFeO3,0.637300,Y,Fe,"(Y, Fe, O)"
7,LaCrO3,1.903900,La,Cr,"(La, Cr, O)"
8,PrCrO3,1.847000,Pr,Cr,"(Pr, Cr, O)"
9,NdCrO3,1.904000,Nd,Cr,"(Nd, Cr, O)"


In [ ]:
non_single_oxide = pd.read_csv('/data/non_single_oxide.csv').drop(columns = 'Unnamed: 0')

In [24]:
single_oxide = small_dataset.drop(columns = ['target','A','B','composition'])
non_single_oxide = non_single_oxide.drop(columns = 'target')

In [25]:
single_oxide['is_single_oxide'] = 1
non_single_oxide['is_single_oxide'] = 0
data = pd.concat([single_oxide,non_single_oxide],axis=0).reset_index(drop=True)
data

,formula,is_single_oxide
0,SmCrO3,1
1,ScCoO3,1
2,GdCrO3,1
3,YCrO3,1
4,LaFeO3,1
...,...,...
107,KSb2PO8,0
108,Sr4Mn2CuO9,0
109,Mg6AlBiO8,0
110,Ba4Sr(SnO3)5,0


In [26]:
from matminer.featurizers.base import MultipleFeaturizer
from matminer.featurizers import composition as cf
feature_calculators = MultipleFeaturizer([cf.ElementProperty.from_preset("magpie"),
                                          cf.ValenceOrbital(props=['avg']), cf.IonProperty(fast=True)])
feature_labels = feature_calculators.feature_labels()
data['composition'] = data['formula'].map(Composition)
data_features = feature_calculators.featurize_dataframe(data,col_id='composition')

MultipleFeaturizer:   0%|          | 0/112 [00:00<?, ?it/s]

In [27]:
def remove_collinear_features(x, threshold):
    '''
    Objective:
        Remove collinear features in a dataframe with a correlation coefficient
        greater than the threshold. Removing collinear features can help a model
        to generalize and improves the interpretability of the model.
        
    Inputs: 
        threshold: any features with correlations greater than this value are removed
    
    Output: 
        dataframe that contains only the non-highly-collinear features
    '''  
    # Calculate the correlation matrix
    corr_matrix = x.corr()
    iters = range(len(corr_matrix.columns) - 1)
    drop_cols = []

    # Iterate through the correlation matrix and compare correlations
    for i in iters:
        for j in range(i):
            item = corr_matrix.iloc[j:(j+1), (i+1):(i+2)]
            col = item.columns
            row = item.index
            val = abs(item.values)
            
            # If correlation exceeds the threshold
            if val >= threshold:
                # Print the correlated features and the correlation value
                # print(col.values[0], "|", row.values[0], "|", round(val[0][0], 2))
                drop_cols.append(col.values[0])

    # Drop one of each pair of correlated columns
    drops = set(drop_cols)
    x = x.drop(columns = drops)
    return x

In [28]:
X = data_features[feature_labels]
y = data['is_single_oxide']
X = remove_collinear_features(X, 0.8)
feature_labels = X.columns

In [31]:
X_train,X_test,y_train,y_test = train_test_split(X,y,shuffle = True,random_state=42)
from sklearn.model_selection import StratifiedKFold
from tpot import TPOTClassifier
cv = StratifiedKFold(n_splits=5) 

model_clf = TPOTClassifier(generations=10,population_size=50,cv=cv, scoring='roc_auc', verbosity=2, 
                       random_state=42, n_jobs=-1) 
model_clf.fit(X_train, y_train)

Version 0.12.1 of tpot is outdated. Version 0.12.2 was released Friday February 23, 2024.


Optimization Progress:   0%|          | 0/550 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9666666666666668

Generation 2 - Current best internal CV score: 0.9666666666666668

Generation 3 - Current best internal CV score: 0.9666666666666668

Generation 4 - Current best internal CV score: 0.9726190476190476

Generation 5 - Current best internal CV score: 0.9726190476190476

Generation 6 - Current best internal CV score: 0.9785714285714286

Generation 7 - Current best internal CV score: 0.9785714285714286

Generation 8 - Current best internal CV score: 0.9785714285714286

Generation 9 - Current best internal CV score: 0.9813492063492063

Generation 10 - Current best internal CV score: 0.9813492063492063

Best pipeline: GradientBoostingClassifier(input_matrix, learning_rate=0.1, max_depth=6, max_features=0.3, min_samples_leaf=9, min_samples_split=19, n_estimators=100, subsample=0.55)


TPOTClassifier(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
               generations=10, n_jobs=-1, population_size=50, random_state=42,
               scoring='roc_auc', verbosity=2)

In [ ]:
best_model = model_clf.fitted_pipeline_

best_model.fit(X_train, y_train)

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score,accuracy_score,f1_score,matthews_corrcoef,classification_report,roc_curve, auc,precision_score,recall_score
y_test_pred = best_model.predict(X_test)

acc = accuracy_score(y_test, y_test_pred)
print(f"accuracy: {acc}")
roc_auc = roc_auc_score(y_test,y_test_pred)
print(f"roc_auc: {roc_auc}")
f1 = f1_score(y_test, y_test_pred ,average='weighted')
print(f"f1: {f1}")

accuracy: 0.9642857142857143
roc_auc: 0.9666666666666667
f1: 0.9643313264002921


In [ ]:
from joblib import dump
model_file_path = 'space_clf_tpot.joblib'
dump(best_model, model_file_path)
print("Model saved successfully at:", model_file_path)

Model saved successfully at: space_clf_tpot.joblib


In [10]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split


X_train,X_test,y_train,y_test = train_test_split(X, y, random_state=42)

# Average CV score on the training set was: 0.9467857142857143
model_clf = GradientBoostingClassifier(learning_rate=0.1, max_depth=2, max_features=0.25, min_samples_leaf=9, min_samples_split=10, n_estimators=100, subsample=0.7500000000000001)
# Fix random state in exported estimator
if hasattr(model_clf, 'random_state'):
    setattr(model_clf, 'random_state', 42)

model_clf.fit(X_train, y_train)

GradientBoostingClassifier(max_depth=2, max_features=0.25, min_samples_leaf=9,
                           min_samples_split=10, random_state=42,
                           subsample=0.7500000000000001)

In [34]:
# from sklearn.metrics import roc_auc_score
# y_test_pred = model_clf.predict(X_test)
# roc_auc = roc_auc_score(y_test,y_test_pred)
# roc_auc

In [ ]:
a_dop_data = pd.read_csv('data/a_replace_data.csv').drop(columns = 'Unnamed: 0')
# b_dop_data = pd.read_csv('b_replace_data.csv').drop(columns = 'Unnamed: 0')

In [ ]:
b_dop_data = pd.read_csv('data/b_replace_data.csv').drop(columns = 'Unnamed: 0')

In [37]:
b_dop_data

,formula,A1,A1_ratio,nA1,A2,A2_ratio,nA2,B1,B1_ratio,B2,B2_ratio,X,X_ratio,Ra,Rb,Rx,nA
0,Sm1Cr0.8Co0.2O3,Sm,1,3,Sm,0,3,Cr,0.8,Co,0.2,O,3,1.24,0.614,1.4,3
1,Sm1Cr0.8Fe0.2O3,Sm,1,3,Sm,0,3,Cr,0.8,Fe,0.2,O,3,1.24,0.621,1.4,3
2,Sm1Nb0.2Cr0.8O3,Sm,1,3,Sm,0,3,Cr,0.8,Nb,0.2,O,3,1.24,0.620,1.4,3
3,Sm1Cr0.8In0.2O3,Sm,1,3,Sm,0,3,Cr,0.8,In,0.2,O,3,1.24,0.652,1.4,3
4,Sm1Er0.2Cr0.8O3,Sm,1,3,Sm,0,3,Cr,0.8,Er,0.2,O,3,1.24,0.670,1.4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43177,SrGaO3,Sr,1,2,Sr,0,2,Ga,1.0,Cr,0.0,O,3,1.44,0.620,1.4,2
43178,SrTaO3,Sr,1,2,Sr,0,2,Ta,1.0,Cr,0.0,O,3,1.44,0.640,1.4,2
43179,SrAlO3,Sr,1,2,Sr,0,2,Al,1.0,Cr,0.0,O,3,1.44,0.535,1.4,2
43180,SrSbO3,Sr,1,2,Sr,0,2,Sb,1.0,Cr,0.0,O,3,1.44,0.600,1.4,2


In [38]:
a_dop_data['composition'] = a_dop_data['formula'].map(Composition)
# b_dop_data['composition'] = b_dop_data['formula'].map(Composition)

In [39]:
b_dop_data['composition'] = b_dop_data['formula'].map(Composition)

In [40]:
a_data_features = feature_calculators.featurize_dataframe(a_dop_data, col_id='composition',ignore_errors=True)

MultipleFeaturizer:   0%|          | 0/64452 [00:00<?, ?it/s]

In [41]:
a_data_features

,formula,A1,A1_ratio,nA1,A2,A2_ratio,nA2,B1,B1_ratio,B2,...,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber,avg s valence electrons,avg p valence electrons,avg d valence electrons,avg f valence electrons,compound possible,max ionic char,avg ionic char
0,Sm0.2Sc0.8Cr1O3,Sm,0.2,3,Sc,0.8,3,Cr,1,Cr,...,90.68,94.416,12.0,1.8,2.4,1.16,0.24,True,0.724240,0.147721
1,Sm0.2Sc0.8Co1O3,Sm,0.2,3,Sc,0.8,3,Co,1,Co,...,83.68,86.016,12.0,2.0,2.4,1.56,0.24,True,0.724240,0.138623
2,Sm0.2Sc0.8Fe1O3,Sm,0.2,3,Sc,0.8,3,Fe,1,Fe,...,90.68,94.416,12.0,2.0,2.4,1.36,0.24,True,0.724240,0.140666
3,Sm0.2Sc0.8Nb1O3,Sm,0.2,3,Sc,0.8,3,Nb,1,Nb,...,90.68,94.416,12.0,1.8,2.4,0.96,0.24,False,0.724240,0.150234
4,Sm0.2Sc0.8Sn1O3,Sm,0.2,3,Sc,0.8,3,Sn,1,Sn,...,73.08,73.296,12.0,2.0,2.8,2.16,0.24,False,0.724240,0.135400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64447,K0.8Rb0.2Ga1O3,K,0.8,1,Rb,0.2,1,Ga,1,Ga,...,65.80,65.280,12.0,1.8,2.6,2.00,0.00,False,0.820234,0.165360
64448,K0.8Rb0.2Ta1O3,K,0.8,1,Rb,0.2,1,Ta,1,Ta,...,98.80,104.160,12.0,1.8,2.4,0.60,2.80,True,0.820234,0.175962
64449,K0.8Rb0.2Bi1O3,K,0.8,1,Rb,0.2,1,Bi,1,Bi,...,55.40,69.440,12.0,1.8,3.0,2.00,2.80,False,0.820234,0.158035
64450,K0.8Rb0.2Sb1O3,K,0.8,1,Rb,0.2,1,Sb,1,Sb,...,86.20,89.040,12.0,1.8,3.0,2.00,0.00,True,0.820234,0.156995


In [42]:
a_dop_data = pd.merge(a_dop_data,a_data_features,on = 'formula')
a_dop_data = a_dop_data.drop_duplicates(subset=['formula'], keep='first').reset_index(drop=True)
a_dop_data

,formula,A1_x,A1_ratio_x,nA1_x,A2_x,A2_ratio_x,nA2_x,B1_x,B1_ratio_x,B2_x,...,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber,avg s valence electrons,avg p valence electrons,avg d valence electrons,avg f valence electrons,compound possible,max ionic char,avg ionic char
0,Sm0.2Sc0.8Cr1O3,Sm,0.2,3,Sc,0.8,3,Cr,1,Cr,...,90.68,94.416,12.0,1.8,2.4,1.16,0.24,True,0.724240,0.147721
1,Sm0.2Sc0.8Co1O3,Sm,0.2,3,Sc,0.8,3,Co,1,Co,...,83.68,86.016,12.0,2.0,2.4,1.56,0.24,True,0.724240,0.138623
2,Sm0.2Sc0.8Fe1O3,Sm,0.2,3,Sc,0.8,3,Fe,1,Fe,...,90.68,94.416,12.0,2.0,2.4,1.36,0.24,True,0.724240,0.140666
3,Sm0.2Sc0.8Nb1O3,Sm,0.2,3,Sc,0.8,3,Nb,1,Nb,...,90.68,94.416,12.0,1.8,2.4,0.96,0.24,False,0.724240,0.150234
4,Sm0.2Sc0.8Sn1O3,Sm,0.2,3,Sc,0.8,3,Sn,1,Sn,...,73.08,73.296,12.0,2.0,2.8,2.16,0.24,False,0.724240,0.135400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64447,K0.8Rb0.2Ga1O3,K,0.8,1,Rb,0.2,1,Ga,1,Ga,...,65.80,65.280,12.0,1.8,2.6,2.00,0.00,False,0.820234,0.165360
64448,K0.8Rb0.2Ta1O3,K,0.8,1,Rb,0.2,1,Ta,1,Ta,...,98.80,104.160,12.0,1.8,2.4,0.60,2.80,True,0.820234,0.175962
64449,K0.8Rb0.2Bi1O3,K,0.8,1,Rb,0.2,1,Bi,1,Bi,...,55.40,69.440,12.0,1.8,3.0,2.00,2.80,False,0.820234,0.158035
64450,K0.8Rb0.2Sb1O3,K,0.8,1,Rb,0.2,1,Sb,1,Sb,...,86.20,89.040,12.0,1.8,3.0,2.00,0.00,True,0.820234,0.156995


In [43]:
b_data_features = feature_calculators.featurize_dataframe(b_dop_data, col_id='composition',ignore_errors=True)

MultipleFeaturizer:   0%|          | 0/43182 [00:00<?, ?it/s]

In [44]:
b_dop_data = pd.merge(b_dop_data,b_data_features,on = 'formula')
b_dop_data = b_dop_data.drop_duplicates(subset=['formula'], keep='first').reset_index(drop=True)
b_dop_data

,formula,A1_x,A1_ratio_x,nA1_x,A2_x,A2_ratio_x,nA2_x,B1_x,B1_ratio_x,B2_x,...,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber,avg s valence electrons,avg p valence electrons,avg d valence electrons,avg f valence electrons,compound possible,max ionic char,avg ionic char
0,Sm1Cr0.8Co0.2O3,Sm,1,3,Sm,0,3,Cr,0.8,Co,...,84.8,87.36,12.0,1.84,2.40,1.08,1.20,True,0.724240,0.153258
1,Sm1Cr0.8Fe0.2O3,Sm,1,3,Sm,0,3,Cr,0.8,Fe,...,86.2,89.04,12.0,1.84,2.40,1.04,1.20,True,0.724240,0.153613
2,Sm1Nb0.2Cr0.8O3,Sm,1,3,Sm,0,3,Cr,0.8,Nb,...,86.2,89.04,12.0,1.80,2.40,0.96,1.20,False,0.724240,0.155368
3,Sm1Cr0.8In0.2O3,Sm,1,3,Sm,0,3,Cr,0.8,In,...,82.6,84.72,12.0,1.84,2.44,1.20,1.20,True,0.724240,0.153978
4,Sm1Er0.2Cr0.8O3,Sm,1,3,Sm,0,3,Cr,0.8,Er,...,84.8,87.36,12.0,1.84,2.40,0.80,1.68,True,0.724240,0.158425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43177,SrGaO3,Sr,1,2,Sr,0,2,Ga,1.0,Cr,...,65.0,64.00,12.0,2.00,2.60,2.00,0.00,False,0.787757,0.159523
43178,SrTaO3,Sr,1,2,Sr,0,2,Ta,1.0,Cr,...,98.0,103.20,12.0,2.00,2.40,0.60,2.80,False,0.787757,0.170611
43179,SrAlO3,Sr,1,2,Sr,0,2,Al,1.0,Cr,...,97.2,102.24,12.0,2.00,2.60,0.00,0.00,False,0.787757,0.166709
43180,SrSbO3,Sr,1,2,Sr,0,2,Sb,1.0,Cr,...,85.4,88.08,12.0,2.00,3.00,2.00,0.00,False,0.787757,0.150942


In [45]:
a_X = a_data_features[feature_labels]
a_dop_data['is_metal_oxide'] = model_clf.predict(a_X)
a_dop_data['is_metal_oxide'].value_counts()

1    45476
0    18976
Name: is_metal_oxide, dtype: int64

In [46]:
b_X = b_data_features[feature_labels]
b_dop_data['is_metal_oxide'] = model_clf.predict(b_X)
b_dop_data['is_metal_oxide'].value_counts()

1    28046
0    15136
Name: is_metal_oxide, dtype: int64

In [47]:
a_data_221 = a_dop_data[a_dop_data['is_metal_oxide']==1]
b_data_221 = b_dop_data[b_dop_data['is_metal_oxide']==1]
print(a_data_221.shape,b_data_221.shape)

(45476, 175) (28046, 175)


In [48]:
feature_calculators = MultipleFeaturizer([cf.ElementProperty.from_preset("magpie"),
                                          cf.ValenceOrbital(props=['avg']), cf.IonProperty(fast=True)])
magpie_feature_labels = feature_calculators.feature_labels()

In [49]:
dop_data_221 = pd.concat([a_data_221,b_data_221],axis=0).reset_index(drop = True)
dop_data_221

,formula,A1_x,A1_ratio_x,nA1_x,A2_x,A2_ratio_x,nA2_x,B1_x,B1_ratio_x,B2_x,...,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber,avg s valence electrons,avg p valence electrons,avg d valence electrons,avg f valence electrons,compound possible,max ionic char,avg ionic char,is_metal_oxide
0,Sm0.2Sc0.8Cr1O3,Sm,0.2,3,Sc,0.8,3,Cr,1.0,Cr,...,94.416,12.0,1.8,2.4,1.16,0.24,True,0.724240,0.147721,1
1,Sm0.2Sc0.8Co1O3,Sm,0.2,3,Sc,0.8,3,Co,1.0,Co,...,86.016,12.0,2.0,2.4,1.56,0.24,True,0.724240,0.138623,1
2,Sm0.2Sc0.8Fe1O3,Sm,0.2,3,Sc,0.8,3,Fe,1.0,Fe,...,94.416,12.0,2.0,2.4,1.36,0.24,True,0.724240,0.140666,1
3,Sm0.2Sc0.8Nb1O3,Sm,0.2,3,Sc,0.8,3,Nb,1.0,Nb,...,94.416,12.0,1.8,2.4,0.96,0.24,False,0.724240,0.150234,1
4,Sm0.2Sc0.8Sn1O3,Sm,0.2,3,Sc,0.8,3,Sn,1.0,Sn,...,73.296,12.0,2.0,2.8,2.16,0.24,False,0.724240,0.135400,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73517,SrSnO3,Sr,1.0,2,Sr,0.0,2,Sn,1.0,Cr,...,82.080,12.0,2.0,2.8,2.00,0.00,True,0.787757,0.154135,1
73518,SrZrO3,Sr,1.0,2,Sr,0.0,2,Zr,1.0,Cr,...,94.800,12.0,2.0,2.4,0.40,0.00,True,0.787757,0.176522,1
73519,SrHfO3,Sr,1.0,2,Sr,0.0,2,Hf,1.0,Cr,...,94.800,12.0,2.0,2.4,0.40,2.80,True,0.787757,0.177547,1
73520,SrTiO3,Sr,1.0,2,Sr,0.0,2,Ti,1.0,Cr,...,94.800,12.0,2.0,2.4,0.40,0.00,True,0.787757,0.169198,1


In [50]:
a_data_221

,formula,A1_x,A1_ratio_x,nA1_x,A2_x,A2_ratio_x,nA2_x,B1_x,B1_ratio_x,B2_x,...,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber,avg s valence electrons,avg p valence electrons,avg d valence electrons,avg f valence electrons,compound possible,max ionic char,avg ionic char,is_metal_oxide
0,Sm0.2Sc0.8Cr1O3,Sm,0.2,3,Sc,0.8,3,Cr,1,Cr,...,94.416,12.0,1.8,2.4,1.16,0.24,True,0.724240,0.147721,1
1,Sm0.2Sc0.8Co1O3,Sm,0.2,3,Sc,0.8,3,Co,1,Co,...,86.016,12.0,2.0,2.4,1.56,0.24,True,0.724240,0.138623,1
2,Sm0.2Sc0.8Fe1O3,Sm,0.2,3,Sc,0.8,3,Fe,1,Fe,...,94.416,12.0,2.0,2.4,1.36,0.24,True,0.724240,0.140666,1
3,Sm0.2Sc0.8Nb1O3,Sm,0.2,3,Sc,0.8,3,Nb,1,Nb,...,94.416,12.0,1.8,2.4,0.96,0.24,False,0.724240,0.150234,1
4,Sm0.2Sc0.8Sn1O3,Sm,0.2,3,Sc,0.8,3,Sn,1,Sn,...,73.296,12.0,2.0,2.8,2.16,0.24,False,0.724240,0.135400,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64444,K0.8Rb0.2Zr1O3,K,0.8,1,Rb,0.2,1,Zr,1,Zr,...,95.760,12.0,1.8,2.4,0.40,0.00,False,0.820234,0.181519,1
64445,K0.8Rb0.2Hf1O3,K,0.8,1,Rb,0.2,1,Hf,1,Hf,...,95.760,12.0,1.8,2.4,0.40,2.80,False,0.820234,0.182476,1
64446,K0.8Rb0.2Ti1O3,K,0.8,1,Rb,0.2,1,Ti,1,Ti,...,95.760,12.0,1.8,2.4,0.40,0.00,False,0.820234,0.174624,1
64448,K0.8Rb0.2Ta1O3,K,0.8,1,Rb,0.2,1,Ta,1,Ta,...,104.160,12.0,1.8,2.4,0.60,2.80,True,0.820234,0.175962,1


In [51]:
# dop_data_221.to_csv('dop_data_221_new.csv')

In [52]:
# a_data_221.to_csv('a_data_221_new.csv')
# b_data_221.to_csv('b_data_221_new.csv')